This notebook has all functions.
Methods :

Method 1 : Threshold-->Filter-->Erosion-->Dilation
Method 2 : Filter-->Threshold-->Erosion-->Dilation

In [1]:
# to save labelled images
BASE_DIR="/Users/Trupti/01-LIDo/02-VijiProject/ImageAnalysis/" 


In [2]:
def save_img(folder_path,img_name,labelled_name):
    LABELLED_IMG_DIR = BASE_DIR + "AnalysisMethods/AnalysisResults/XMovie/set2-pp1/labelled_images/"
    directory=folder_path.split('/')[-1].split('.')[0] # to create a folder per experiment to save images
    path = LABELLED_IMG_DIR  + directory
    try:
        os.makedirs(path)
    except FileExistsError:
    # directory already exists
        pass
    plt.imsave((path + '/' +'labelled'+'_'+ labelled_name +'.png'),img_name,dpi=300)
    

In [3]:
def cytoplasm_signal(img):
  '''
  This function takes an 8bit image,calculates the pixel value for all 4 corners in a 5x5 window 
  and returns its mean.
  '''
  col,row=img.shape
  topLeft=img[0:10, 0:10].flatten() 
  topRight=img[col-10:col,0:10].flatten()
  bottomLeft=img[0:10,col-10:col].flatten() 
  bottomRight=img[col-10:col,col-10:col].flatten()

  mean_array=np.concatenate([topLeft,topRight,bottomLeft,bottomRight])
  mean=np.mean(mean_array)
  
  return(mean)

In [ ]:
def mask(maskimg,maskim,propList):
    '''
    This function calculates the intensity of Nuf2-GFP on KT
    Image Used : Mask Image
    
    '''
    #create a mask for the particles
    #Apply threshold
    threshold = filters.threshold_yen(maskimg)
    #Generate thresholded image
    threshold_image = maskimg > threshold
    # Apply prewitt filter to threshold image
    prewitt_im= filters.prewitt(threshold_image)
        #Apply erosion to the filtered image followed by dilation to the eroded image

    erosion_im=morphology.binary_erosion(prewitt_im, selem=None, out=None)
    dilation_im=morphology.binary_dilation(erosion_im, selem=None, out=None)

    labelled_mask,num_labels=ndi.label(dilation_im)


    all_props=measure.regionprops_table(labelled_mask, intensity_image=maskim, properties=propList)#['label','area','min_intensity','max_intensity','mean_intensity']) # intensity image is 16 bit red channel image
    df_mask= pd.DataFrame(all_props)
    return(df_mask,dilation_im,labelled_mask)

In [ ]:
def channel(particle_mask,channel_img,channel_im,propList):
    '''
    This function calculates the astrin intensity on KT
   
    '''
    # find the co-ordinates of the rest of the image without the particles
    # dilation_im is the binary image with particles 1 and bg 0

    rest_of_img_idx=np.where(particle_mask==0)
    # copy channel image into another for further operations
    channel_channel_img= channel_img.copy()
    channel_channel_img[rest_of_img_idx]=0
    # create a mask for the channel after removing the background. We are interested only in the 
    # particles
    threshold=filters.threshold_triangle(channel_channel_img)
    channel_mask =channel_channel_img> threshold # main cell stays black
    # find the intensity of the particles(ASTRIN) without the cytoplasm
    labelled_channel_mask,num_labels_channel=ndi.label(channel_mask)
    all_props=measure.regionprops_table(labelled_channel_mask, intensity_image=channel_im, properties=propList)#['label','area','min_intensity','max_intensity','mean_intensity']) # intensity image is 16 bit red channel image
    df_channel= pd.DataFrame(all_props)
    return(df_channel)

In [ ]:
def mask_cyto_fun(particle_mask,maskimg,maskim):
    '''
    Nuf2-Cyto : Nuf2-GFP cytoplasmic intensity
    Intensity of Nuf2 signal cell without the particles (mask image)
    '''
    threshold=filters.threshold_mean(maskimg)
    mask_cyto_mask =maskimg > threshold
    mask_cyto_mask = morphology.remove_small_objects(mask_cyto_mask)
    mask_cyto_mask = morphology.remove_small_holes(mask_cyto_mask)
    # black idx we need to remove the particles. hence select the particles idx 
    idx = np.where(particle_mask != 0)
    mask_cyto_mask[idx]=0 
    labelled_mask_cyto_mask,num_labels_mask_cyto=ndi.label(mask_cyto_mask)
    all_props=measure.regionprops_table(labelled_mask_cyto_mask, intensity_image=maskim, properties=['label','area','min_intensity','max_intensity','mean_intensity']) # intensity image is 16 bit red channel image
    df_mask_cyto= pd.DataFrame(all_props)
    
    try:
        mask_cyto=df_mask_cyto[df_mask_cyto['area']==df_mask_cyto['area'].max()]['mean_intensity'].values[0]
        return(mask_cyto)
    except:
        return(np.nan)
    

    

In [1]:
def channel_cyto_fun_pp1(particle_mask,channel_img,channel_im):
    
    '''
    Astrin Cyto : mKate2-Astrin cytoplasmic intensity
    Intensity of mKate2-Astrin signal cell without the particles (channel image)
    
    '''
    # mask for the cell
    threshold=filters.threshold_mean(channel_img)

    channel_cyto_mask =channel_img >threshold
    channel_cyto_mask = morphology.remove_small_objects(channel_cyto_mask)
    channel_cyto_mask = morphology.remove_small_holes(channel_cyto_mask)
    
     #black idx # remove KTs

    kt_idx = np.where(particle_mask != 0)
    channel_cyto_mask[kt_idx]=0 
                                                     
    labelled_channel_cyto_mask,num_labels_channel_cyto=ndi.label(channel_cyto_mask)
    all_props=measure.regionprops_table(labelled_channel_cyto_mask, intensity_image=channel_im, properties=['label','area','mean_intensity']) # intensity image is 16 bit red channel image
    df_channel_cyto= pd.DataFrame(all_props)

    try:
        channelcyto=df_channel_cyto[df_channel_cyto['area']==df_channel_cyto['area'].max()]['mean_intensity'].values[0]
        return(channelcyto)
    except:
        return(np.nan)
    
     

In [2]:
def channel_cyto_fun_astrin(particle_mask,channel_img,channel_im):
    
    '''
    Astrin Cyto : mKate2-Astrin cytoplasmic intensity
    Intensity of mKate2-Astrin signal cell without the particles (channel image)
    
    '''
    # mask for the cell
    threshold=filters.threshold_mean(channel_img)

    channel_cyto_mask =channel_img >threshold
    channel_cyto_mask = morphology.remove_small_objects(channel_cyto_mask)
    channel_cyto_mask = morphology.remove_small_holes(channel_cyto_mask)
    # find microtubule

    triangle_threshold=filters.threshold_triangle(channel_img)
    microtubule=channel_img < triangle_threshold

    # remove microtubule not needed for pp1
    mt_idx = np.where(microtubule== 0)
    channel_cyto_mask[mt_idx]=0
    #black idx # remove KTs

    kt_idx = np.where(particle_mask != 0)
    channel_cyto_mask[kt_idx]=0 
                                                     
    labelled_channel_cyto_mask,num_labels_channel_cyto=ndi.label(channel_cyto_mask)
    all_props=measure.regionprops_table(labelled_channel_cyto_mask, intensity_image=channel_im, properties=['label','area','mean_intensity']) # intensity image is 16 bit red channel image
    df_channel_cyto= pd.DataFrame(all_props)

    try:
        channelcyto=df_channel_cyto[df_channel_cyto['area']==df_channel_cyto['area'].max()]['mean_intensity'].values[0]
        return(channelcyto)
    except:
        return(np.nan)
    

In [1]:
def method_pp1(folder_path):
    '''
      This function takes the folder path of tif images and performs following steps.
      1. Reads the image from the path
      2. Converts the 16bit image to 8 bit
      3. Prewitt Filter-->Yen Threshold-->Erosion-->dilation
  
      For mean intensity calculation, the background noise needs to be filtered from the intensity image. 

    '''
    propList = ['label', 'mean_intensity'] 
    df_mask_final = pd.DataFrame(columns=propList)
    df_channel_final= pd.DataFrame(columns=propList)
    
    # set path for images
    channel_chpath = os.path.join(folder_path,"channel","*.tif") #C1 red channel
    mask_chpath = os.path.join(folder_path,"mask","*.tif") #C0 green channel
    # create red channel image array
    channel_image_list=[]
    for file in natsorted(glob.glob(channel_chpath)):
        channel_image_list.append(file)
     
    k=0
    for file in natsorted(glob.glob(mask_chpath)):
        mask_im= io.imread(file)  # This is to measure and label the particles
        #Convert an (ImageJ) TIFF to an 8 bit numpy array
        mask_img= (mask_im / np.amax(mask_im) * 255).astype(np.uint8)
        channel_im = io.imread(channel_image_list[k])
        channel_img= (channel_im / np.amax(channel_im) * 255).astype(np.uint8)
        
        # get nuf2 signal
        df_mask,particle_mask,labelled_mask=mask(mask_img,mask_im,propList)
        mask_fname=re.sub(r'^.+/([^/]+)$', r'\1',file)
        #df_nuf2.insert(0,'nuf2_fname',nuf_fname)
        df_mask['label']=str(k) +"_"+ df_mask['label'].astype(str)
        df_mask['fname']=re.sub(r'^.+/([^/]+)$', r'\1',file)
        
       
        # get astrin signal
        df_channel=channel(particle_mask,channel_img,channel_im,propList)        
        as_fname=re.sub(r'^.+/([^/]+)$', r'\1',channel_image_list[k])
        #df_astrin.insert(0,'astrin_fname',as_fname)
        df_channel['fname']=re.sub(r'^.+/([^/]+)$', r'\1',channel_image_list[k])
        df_channel['label']=str(k) +"_"+ df_channel['label'].astype(str)
        
        # cytoplasm intensity of mask image
        mask_cyto=mask_cyto_fun(particle_mask,mask_img,mask_im)
        #cytoplasm intensity for channel image
        channel_cyto=channel_cyto_fun_pp1(particle_mask,channel_img,channel_im)
        # mask bg outside the cell
        mask_bg=cytoplasm_signal(mask_im)
        # channel bg outside the cell
        channel_bg=cytoplasm_signal(channel_im)
        
        df_mask_final=pd.concat([df_mask_final,df_mask],ignore_index=True)        
        df_channel_final=pd.concat([df_channel_final,df_channel],ignore_index=True)
        
        
        # #SAVE THE IMAGES : uncomment to save the images
        labelled_name=re.sub(r'^.+/([^/]+)$', r'\1',file)
        
        save_img(folder_path,labelled_mask,labelled_name)
       
       
        
        k+=1
  
    return(df_mask_final,df_channel_final,mask_cyto,channel_cyto,mask_bg,channel_bg)

In [ ]:
def method_astrin(folder_path):
    '''
      This function takes the folder path of tif images and performs following steps.
      1. Reads the image from the path
      2. Converts the 16bit image to 8 bit
      3. Prewitt Filter-->Yen Threshold-->Erosion-->dilation
  
      For mean intensity calculation, the background noise needs to be filtered from the intensity image. 

    '''
    propList = ['label', 'mean_intensity'] 
    df_mask_final = pd.DataFrame(columns=propList)
    df_channel_final= pd.DataFrame(columns=propList)
    
    # set path for images
    channel_chpath = os.path.join(folder_path,"channel","*.tif") #C1 red channel
    mask_chpath = os.path.join(folder_path,"mask","*.tif") #C0 green channel
    # create red channel image array
    channel_image_list=[]
    for file in natsorted(glob.glob(channel_chpath)):
        channel_image_list.append(file)
     
    k=0
    for file in natsorted(glob.glob(mask_chpath)):
        mask_im= io.imread(file)  # This is to measure and label the particles
        #Convert an (ImageJ) TIFF to an 8 bit numpy array
        mask_img= (mask_im / np.amax(mask_im) * 255).astype(np.uint8)
        channel_im = io.imread(channel_image_list[k])
        channel_img= (channel_im / np.amax(channel_im) * 255).astype(np.uint8)
        
        # get nuf2 signal
        df_mask,particle_mask,labelled_mask=mask(mask_img,mask_im,propList)
        mask_fname=re.sub(r'^.+/([^/]+)$', r'\1',file)
        #df_nuf2.insert(0,'nuf2_fname',nuf_fname)
        df_mask['label']=str(k) +"_"+ df_mask['label'].astype(str)
        df_mask['fname']=re.sub(r'^.+/([^/]+)$', r'\1',file)
        
       
        # get astrin signal
        df_channel=channel(particle_mask,channel_img,channel_im,propList)        
        as_fname=re.sub(r'^.+/([^/]+)$', r'\1',channel_image_list[k])
        #df_astrin.insert(0,'astrin_fname',as_fname)
        df_channel['fname']=re.sub(r'^.+/([^/]+)$', r'\1',channel_image_list[k])
        df_channel['label']=str(k) +"_"+ df_channel['label'].astype(str)
        
        # cytoplasm intensity of mask image
        mask_cyto=mask_cyto_fun(particle_mask,mask_img,mask_im)
        #cytoplasm intensity for channel image
        channel_cyto=channel_cyto_fun_astrin(particle_mask,channel_img,channel_im)
        # mask bg outside the cell
        mask_bg=cytoplasm_signal(mask_im)
        # channel bg outside the cell
        channel_bg=cytoplasm_signal(channel_im)
        
        df_mask_final=pd.concat([df_mask_final,df_mask],ignore_index=True)        
        df_channel_final=pd.concat([df_channel_final,df_channel],ignore_index=True)
        
        
        # #SAVE THE IMAGES : uncomment to save the images
        labelled_name=re.sub(r'^.+/([^/]+)$', r'\1',file)
        
        save_img(folder_path,labelled_mask,labelled_name)
       
       
        
        k+=1
  
    return(df_mask_final,df_channel_final,mask_cyto,channel_cyto,mask_bg,channel_bg)